In [62]:
import csv
import pandas as pd

In [84]:
class Effect:
    all_effects = []
    bath = pd.read_pickle('data/bath_pickle.pkl')

    def __init__(self, description: str, lower_threshold: float, upper_threshold: float, cost_equation: str, units: str):
        self.description = description
        self.lower_threshold = lower_threshold
        self.upper_threshold = upper_threshold
        self.cost_equation = cost_equation
        self.units = units
        Effect.all_effects.append(self)

    def cost_function(self, elevation_m: float) -> float:
        if self.cost_equation == '':
            return 0
        MAX_SURFACE_AREA = 5574.65
        exposed_km2 = MAX_SURFACE_AREA - Effect.bath.at[round(elevation_m,2), 'Surface Area']
        return eval(self.cost_equation.split('=')[1])
    
    def filled_description(self, elevation: float) -> str:
        cost = self.format_cost_to_print(self.cost_function(elevation))
        return self.description.format(cost=cost)
    
    @staticmethod
    def format_cost_to_print(cost: float) -> str:
        if cost > 1000000000:
            word = 'billion'
            cost /= 1000000000
            return f'${cost:.1f} {word}'
        elif cost > 1000000:
            word = 'million'
            cost /= 1000000            
            return f'${cost:.1f} {word}'
        elif cost > 1000:
            return f'${round(cost/1000)},000'
        else:
            return f'${cost:.0f}'
    
    @classmethod
    def instantiate_from_csv(cls, path:str):
        with open(path, 'r') as f:
            reader = csv.DictReader(f)
            effects = list(reader)
        for effect in effects:
            Effect(
                description = effect.get('\ufeffeffect_description'),
                lower_threshold = float(effect.get('lower_m')),
                upper_threshold = float(effect.get('upper_m')),
                cost_equation = effect.get('cost_equation'),
                units = effect.get('units')
            )

In [91]:
Effect.instantiate_from_csv('data/elevation_effects.csv')

Effect.all_effects[11].filled_description(127)

High salinity impairs the growth of brine shrimp and the algea that brine shrimp eat, harming the brine shrimp industry to the tune of {cost} per year year.
High salinity impairs the growth of brine shrimp and the algea that brine shrimp eat, harming the brine shrimp industry to the tune of $32.3 million per year year.
